In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import re
from matplotlib import pyplot as plt
from tqdm import tqdm

In [6]:
df = pd.read_csv('cleaned-data/test-fixed.csv', header=0, usecols=[0,1])
df.head()

,mr,ref
0,"name[Blue Spice], eatType[coffee shop], area[c...",A coffee shop in the city centre area called B...
1,"name[Blue Spice], eatType[coffee shop], area[c...",Blue Spice is a coffee shop in city centre.
2,"name[Blue Spice], eatType[coffee shop], area[r...",There is a coffee shop Blue Spice in the river...
3,"name[Blue Spice], eatType[coffee shop], area[r...","At the riverside, there is a coffee shop calle..."
4,"name[Blue Spice], eatType[coffee shop], custom...",The coffee shop Blue Spice is based near Crown...


In [3]:
df.iloc[0, :]

mr     name[Blue Spice], eatType[coffee shop], area[c...
ref    A coffee shop in the city centre area called B...
Name: 0, dtype: object

In [9]:
for i in tqdm(range(len(df))):
    mr, text = df.iloc[i,:]
    if i==10: break

  0%|          | 10/4693 [00:00&lt;00:01, 2442.24it/s]


In [4]:
train_data = tf.data.experimental.CsvDataset(filenames='cleaned-data/train-fixed.no-ol.csv', 
                                record_defaults=[tf.string, tf.string],
                                header=True,
                                select_cols=[0, 1])

In [ ]:
for i, d in enumerate(train_data):
    if i > 1:
        break
    print(d)

In [12]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("t5-small")

special_tokens = {'additional_special_tokens': ['<area>', '<eatType>', '<food>', '<near>',                                                      '<name>', '<customer rating>', '<priceRange>',                                                   '<familyFriendly>', '<notfamilyFriendly>',                                                       '<cr_slot>', '<pr_slot>', '<sos>']}
tokenizer.add_special_tokens(special_tokens)

12

In [14]:
tokenizer.convert_tokens_to_ids('</s>')

32111

In [54]:
def p(x):
    seq = tf.cast(tf.logical_not(tf.math.equal(x, 0)), tf.float32)
    return seq[:, :, tf.newaxis]

a = tf.constant([[1, 2, 0, 0], [1, 2, 9, 0],[1, 0, 0, 0]])
b = tf.constant([[6, 5, 3, 8], [6, 5, 3, 8], [6, 5, 3, 8]])
init = tf.keras.initializers.Constant(tf.ones((10, 4), tf.float32))
emb = tf.keras.layers.Embedding(10, 4, embeddings_initializer=init)

print(emb(b) + tf.multiply(emb(a), p(a)))

tf.Tensor(
[[[2. 2. 2. 2.]
  [2. 2. 2. 2.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[2. 2. 2. 2.]
  [2. 2. 2. 2.]
  [2. 2. 2. 2.]
  [1. 1. 1. 1.]]

 [[2. 2. 2. 2.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]], shape=(3, 4, 4), dtype=float32)


In [51]:
p(a)

<tf.Tensor: shape=(3, 4, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [1.],
        [1.]],

       [[0.],
        [0.],
        [0.],
        [1.]],

       [[0.],
        [1.],
        [1.],
        [1.]]], dtype=float32)>

In [28]:
a.dtype

tf.int32

In [11]:
!python train.py -train_path cleaned-data/train-fixed.no-ol.csv -epoch 50 -b 128 -buffer 10000

2020-10-06 19:41:18.716051: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
2020-10-06 19:41:26.750148: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library nvcuda.dll
2020-10-06 19:41:26.750885: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: GeForce MX250 computeCapability: 6.1
coreClock: 1.582GHz coreCount: 3 deviceMemorySize: 2.00GiB deviceMemoryBandwidth: 52.21GiB/s
2020-10-06 19:41:26.751543: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
2020-10-06 19:41:26.751882: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cublas64_10.dll
2020-10-06 19:41:26.752374: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cufft64_10.dll
2020-10